In [ ]:
from google.colab import files
myfiles = files.upload()

Saving slot_filling_output_.csv to slot_filling_output_.csv


In [ ]:
import pandas as pd
import numpy as np
import time
import json

from ast import literal_eval

In [ ]:
output_df = pd.read_csv('slot_filling_output_.csv')
output_df.head()

,cnslt_id,dialogue,ontology & slot,processed,gpt_output
0,4380535,"상담사: 안녕하세요, 고객님. 쏘카 상담사 정소라 입니다. 무엇을 안내해드릴까요?\...",NaN,{},{}
1,4380535,상담사: 환불 관련하여 문의주시어 정확한 상담을 위해 정보 확인 후 안내 도와드리도...,본인확인 : True,{'본인확인': True},{'본인확인': True}
2,4380535,"상담사: 소중한 정보 확인 감사합니다. 많이 궁금하셨을텐데요, 확인 후 안내드리도록...",NaN,{'본인확인': True},{'본인확인': True}
3,4380535,상담사: 기다려주셔서 감사합니다.\n 고객: .,NaN,NaN,{'본인확인': True}
4,4380535,상담사: 고객님 환불 된 부분으로 확인되어 확인되어 번거로우시겠지만 재확인 부탁드립...,환불여부 : False,{'본인확인': True},{'본인확인': True}


In [ ]:
from typing import Any, Dict, List, Sequence, Tuple

def wos_jga(
    pred_steps: Sequence[Sequence[str]], trgt_steps: Sequence[Sequence[str]]
) -> Any:
    total, joint_goal_acc = 0, 0
    for pred_batch, trgt_batch in zip(pred_steps, trgt_steps):
        for pred, trgt in zip(pred_batch, trgt_batch):
            if set(pred) == set(trgt):
                joint_goal_acc += 1
            total += 1

    joint_goal_acc_score = joint_goal_acc / float(total) if total != 0 else 0
    return joint_goal_acc_score * 100.0

In [51]:
# measure JGA
# if gpt_output type is occur error, replace value to 'tmp':'tmp'.

not_matched = 0
all_count = 0
not_matched_list = list()

consult_id = 0
flag = 0

output_data = []
gt_data = []

gpt_turn_list = []
gt_turn_list = []

last_idx = output_df['cnslt_id'].iloc[-1]
print('last index is : ', last_idx)

for idx in range(len(output_df)):
    consult = output_df.iloc[idx]

    if idx == 0:
      consult_id = consult['cnslt_id']
      flag = 1

    elif consult_id != consult['cnslt_id']: # 이전 상담이랑 id가 다르면 turn_list init
      output_data.append(gpt_turn_list)
      gt_data.append(gt_turn_list)

      gpt_turn_list = []
      gt_turn_list = []

      consult_id = consult['cnslt_id']

    elif last_idx == consult['cnslt_id']: # 마지막 행일 경우 output list에 추가
      output_data.append(gpt_turn_list)
      gt_data.append(gt_turn_list)


    if consult['gpt_output'][-1] == "'" :
        consult['gpt_output'] = consult['gpt_output'][:-1]

    consult['gpt_output'] = consult['gpt_output'].replace("null", "None").replace("'",'"')

    if 'false' in consult['gpt_output'] :
        consult['gpt_output'] = consult['gpt_output'].replace('false', 'False')
    if 'true' in consult['gpt_output'] :
        consult['gpt_output'] = consult['gpt_output'].replace('true', 'True')

    if '{"결제 및 환불": {' in consult['gpt_output'] :
        try:
            pred = literal_eval(consult['gpt_output'])['결제 및 환불']
        except:
            gpt_turn_list.append("'tmp':'tmp'")

        try:
          gt = literal_eval(consult['processed'])
        except: # nan 인 경우 handling
          gt = {}

    else:
        try:
            pred = literal_eval(consult['gpt_output'])
        except:
            gpt_turn_list.append("'tmp':'tmp'")

        try:
          gt = literal_eval(consult['processed'])
        except: # nan 인 경우 handling
          gt = {}


    if pred == None or gt == None:
        continue
    else:
        gpt_turn_list.extend([x + '-' + str(y) for x, y in pred.items()])
        gt_turn_list.extend([x + '-' + str(y) for x, y in gt.items()])

print(len(output_data))
print(len(gt_data))

jga_score = wos_jga(output_data, gt_data)
print(jga_score)

last index is :  4319753


<ipython-input-51-790bc634f5e3>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consult['gpt_output'] = consult['gpt_output'].replace("null", "None").replace("'",'"')
<ipython-input-51-790bc634f5e3>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consult['gpt_output'] = consult['gpt_output'][:-1]
<ipython-input-51-790bc634f5e3>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consult['gpt_output'] = consult['gpt_output'].replace('

227
227
27.398081534772185


### measure ROUGE

In [ ]:
!pip install rouge-score

In [52]:
# measure rouge

from rouge_score import rouge_scorer

not_matched = 0
all_count = 0
not_matched_list = list()

consult_id = 0
flag = 0

output_data = []
gt_data = []

gpt_turn_list = []
gt_turn_list = []

last_idx = output_df['cnslt_id'].iloc[-1]
print('last index is : ', last_idx)

for idx in range(len(output_df)):
    consult = output_df.iloc[idx]

    if idx == 0:
      consult_id = consult['cnslt_id']
      flag = 1

    elif consult_id != consult['cnslt_id']: # 이전 상담이랑 id가 다르면 turn_list init
      output_data.append(gpt_turn_list)
      gt_data.append(gt_turn_list)

      gpt_turn_list = []
      gt_turn_list = []

      consult_id = consult['cnslt_id']

    elif last_idx == consult['cnslt_id']: # 마지막 행일 경우 output list에 추가
      output_data.append(gpt_turn_list)
      gt_data.append(gt_turn_list)


    if consult['gpt_output'][-1] == "'" :
        consult['gpt_output'] = consult['gpt_output'][:-1]

    consult['gpt_output'] = consult['gpt_output'].replace("null", "None").replace("'",'"')

    if 'false' in consult['gpt_output'] :
        consult['gpt_output'] = consult['gpt_output'].replace('false', 'False')
    if 'true' in consult['gpt_output'] :
        consult['gpt_output'] = consult['gpt_output'].replace('true', 'True')

    if '{"결제 및 환불": {' in consult['gpt_output'] :
        try:
            pred = literal_eval(consult['gpt_output'])['결제 및 환불']
        except:
            gpt_turn_list.append("'tmp':'tmp'")

        try:
          gt = literal_eval(consult['processed'])
        except: # nan 인 경우 handling
          gt = {}

    else:
        try:
            pred = literal_eval(consult['gpt_output'])
        except:
            gpt_turn_list.append("'tmp':'tmp'")

        try:
          gt = literal_eval(consult['processed'])
        except: # nan 인 경우 handling
          gt = {}


    if pred == None or gt == None:
        continue
    else:
        sorted_pred = dict(sorted(pred.items()))
        sorted_gt = dict(sorted(gt.items()))

        gpt_turn_list.extend([x + '-' + str(y) for x, y in sorted_pred.items()])
        gt_turn_list.extend([x + '-' + str(y) for x, y in sorted_gt.items()])

print(len(output_data))
print(len(gt_data))

total_score = {'rouge1':[0,0,0], 'rouge2':[0,0,0], 'rougeL':[0,0,0]}

for o, g in zip(output_data, gt_data):
  if len(o) == 0:
    o.append('-')
  if len(g) == 0:
    g.append('-')

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
  scores = scorer.score(' '.join(g), ' '.join(o))
  for key in scores:
      total_score[key][0] += scores[key][0] # precision
      total_score[key][1] += scores[key][1] # recall
      total_score[key][2] += scores[key][2] # fmeasure


  # score = bleu.sentence_bleu(g[0].split('-'), o[0].split('-'))
  # total_score += score*100
  # print(score)
  # print(g)
  # print(o)
  # print('-------------')
  # assert 1==2

for key in ['rouge1', 'rouge2', 'rougeL']:
  total_score[key][0] /= len(output_data)
  total_score[key][1] /= len(output_data)
  total_score[key][2] /= len(output_data)


print(total_score['rouge1'])
print(total_score['rouge2'])
print(total_score['rougeL'])


last index is :  4319753


<ipython-input-52-36636906a530>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consult['gpt_output'] = consult['gpt_output'].replace("null", "None").replace("'",'"')
<ipython-input-52-36636906a530>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consult['gpt_output'] = consult['gpt_output'][:-1]
<ipython-input-52-36636906a530>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consult['gpt_output'] = consult['gpt_output'].replace('

227
227
[0.5736028850365733, 0.7893326550621259, 0.6014809928253245]
[0.442204282512756, 0.5474268246169641, 0.45346839866055294]
[0.5735089261320416, 0.7893042633670304, 0.601438823113157]
